## Import Libraries

In [2]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from vit_pytorch.efficient import ViT

In [3]:
print(f"Torch: {torch.__version__}")

Torch: 1.11.0


In [4]:
# Training settings
batch_size = 16
epochs = 30
lr = 3e-5
gamma = 0.7
seed = 42

device = 'cuda'

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

## Load Data

In [6]:
none_crash_tensor_list = glob.glob(os.path.join('None-crash_tensor','*.pickle'))
vulner_tensor_list = glob.glob(os.path.join('Vulner_tensor', '*.pickle'))

none_crash_labels = ['None-crash' for _ in none_crash_tensor_list]
vulner_labels = ['Vulner' for _ in vulner_tensor_list]

In [8]:
print(f"#None-crash data: {len(none_crash_tensor_list)}")
print(f"#Vulner data: {len(vulner_tensor_list)}")

#None-crash data: 30498
#Vulner data: 3744


## Train and Test split

In [11]:
none_crash_train_list, none_crash_test_list = train_test_split(none_crash_tensor_list, 
                                                                test_size=0.1,
                                                                stratify=none_crash_labels,
                                                                random_state=seed)

vulner_train_list, vulner_test_list = train_test_split(vulner_tensor_list, 
                                                        test_size=0.1,
                                                        stratify=vulner_labels,
                                                        random_state=seed)

In [13]:
print("#None-crash train data:", len(none_crash_train_list))
print("#None-crash test data:", len(none_crash_test_list))
print("#Vulner train data:", len(vulner_train_list))
print("#Vunler test data:", len(vulner_test_list))

#None-crash train data: 27448
#None-crash test data: 3050
#Vulner train data: 3369
#Vunler test data: 375


### None-crash + Vulner => Train / Test

In [41]:
train_list = none_crash_train_list + vulner_train_list
test_list = none_crash_test_list + vulner_test_list
random.shuffle(test_list)

### Train => Train / Valid

In [42]:
train_labels = [path.split('\\')[0][:-7] for path in train_list]

In [43]:
train_list, valid_list = train_test_split(train_list, 
                                          test_size=0.2,
                                          stratify=train_labels,
                                          random_state=seed)

In [44]:
print(f"#Train data: {len(train_list)}")
print(f"#Validation data: {len(valid_list)}")
print(f"#Test data: {len(test_list)}")

Train Data: 24653
Validation Data: 6164
Test Data: 3425


## Load Dataset

In [45]:
class lidar_dataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list # 파일명이 들어가야 함

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        lidar_path = self.file_list[idx]
        
        with open(lidar_path,"rb") as fr:
            data = pickle.load(fr)

        tensor = data['tensor']
        tensor = torch.from_numpy(tensor)
        tensor = tensor.to(torch.float32)
        label = data['label']
            
        return tensor, label

In [46]:
train_data = lidar_dataset(train_list)
valid_data = lidar_dataset(valid_list)
test_data = lidar_dataset(test_list)

(tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            2.5000e+01,  1.3000e+01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            2.3000e+01,  1.4000e+01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            2.8000e+01,  1.4000e+01],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            1.5000e+01,  2.9000e+01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            1.2000e+01,  2.8000e+01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            1.0000e+01,  2.5000e+01]],
 
         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           -4.0457e+01, -4.0510e+01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           -3.7587e+01, -3.7451e+01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           -3.4353e+01, -3.4404e+01],
          ...,
    

In [48]:
train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=True)

## Effecient Attention

### Linformer

In [51]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=196+1, #patches + 1
    depth=12,
    heads=8,
    k=64
)

### ViT

In [52]:
model = ViT(
    dim=128,
    image_size=28,
    patch_size=2,
    num_classes=2,
    transformer=efficient_transformer,
    channels=14,
).to(device)

### Training

In [53]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [54]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

  0%|          | 0/1541 [00:00<?, ?it/s]

Epoch : 1 - loss : 0.1497 - acc: 0.9480 - val_loss : 0.0691 - val_acc: 0.9806



  0%|          | 0/1541 [00:00<?, ?it/s]

Epoch : 2 - loss : 0.0580 - acc: 0.9842 - val_loss : 0.0473 - val_acc: 0.9887



  0%|          | 0/1541 [00:00<?, ?it/s]

...

Epoch : 28 - loss : 0.0000 - acc: 1.0000 - val_loss : 0.0303 - val_acc: 0.9972



  0%|          | 0/1541 [00:00<?, ?it/s]

Epoch : 29 - loss : 0.0000 - acc: 1.0000 - val_loss : 0.0316 - val_acc: 0.9972



  0%|          | 0/1541 [00:00<?, ?it/s]

Epoch : 30 - loss : 0.0000 - acc: 1.0000 - val_loss : 0.0331 - val_acc: 0.9972



In [55]:
model.eval()

ViT(
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=2, p2=2)
    (1): Linear(in_features=56, out_features=128, bias=True)
  )
  (transformer): Linformer(
    (net): SequentialSequence(
      (layers): ModuleList(
        (0): ModuleList(
          (0): PreNorm(
            (fn): LinformerSelfAttention(
              (to_q): Linear(in_features=128, out_features=128, bias=False)
              (to_k): Linear(in_features=128, out_features=128, bias=False)
              (to_v): Linear(in_features=128, out_features=128, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
              (to_out): Linear(in_features=128, out_features=128, bias=True)
            )
            (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          )
          (1): PreNorm(
            (fn): FeedForward(
              (w1): Linear(in_features=128, out_features=512, bias=True)
              (act): GELU()
              (dropout): Dr

# Model Test Accuracy

In [108]:
test_data_tensors = [test_data[i][0] for i in range(len(test_data))]
test_data_labels = [test_data[i][1] for i in range(len(test_data))]

In [116]:
correction = 0
wrong = 0
for i in range(len(test_data)):
    test_tensor = np.asarray(test_data_tensors[i])
    test_tensor = np.asarray([test_tensor])
    test_tensor = torch.from_numpy(test_tensor)
    test_tensor = test_tensor.to(torch.float32).cuda()

    pred = model(test_tensor)
    if pred[0][0] > pred[0][1]:
        pred_idx = 0
    elif pred[0][0] < pred[0][1]:
        pred_idx = 1
    else:
        assert pred[0][0] == pred[0][1], 'Same class score'

    if pred_idx == test_data_labels[i]:
        correction = correction + 1
    else:
        wrong = wrong + 1


In [120]:
accuracy = correction/len(test_data)
print("Model Accuracy:", accuracy)

0.9967883211678832

# Model Save

In [14]:
torch.save(model, "model.pt")

NameError: name 'model' is not defined